In [ ]:
#Installing the dependencies
!pip install PyPDF2
!pip install InstructorEmbedding

In [ ]:
#Import necessary Libraries
import torch
import PyPDF2
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader
import os
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)

Loading the docs through PyPDFLoader

In [ ]:

path = '/Users/srikar/Documents/DSRag/data'
os.listdir(path)

['UltimateGuidetoDataScienceInterviews-2.pdf',
 "A Beginner's guide to R.pdf",
 'M1-Machine-Learning-Tom-Mitchell.pdf',
 'M2-Machine_Learning_By_Ethem_Alpaydin.pdf',
 'ML questions_part2.pdf',
 'Deep Learning-Ian Goodfellow.pdf',
 '[Joel_Grus]_Data_Science_from_Scratch_First_Princ.pdf',
 'Data Scientist Interview Questions.pdf',
 'bigdata-analysis-interview-questions.pdf',
 'pythondatasciencehandbook.pdf',
 'Python_Machine_Learning_Sebastian_Raschka.pdf',
 'thinkstats2.pdf',
 'R programming for data science.pdf',
 'Handbook_of_Research_on_Emerging_Trends_and_Applications_of_Machine.pdf',
 'Think_Like_a_Data_Scientist.pdf',
 'Practical_Machine_Learning_with_AWS_Process,_Himanshu Singh.pdf',
 'Prediction Machines-The Simple Economics of Artificial Intelligence by Ajay Agrawal.pdf',
 'python_interview_questions.pdf',
 'Advanced Applied Deep Learning-Umberto Michelucci.pdf',
 'Python for Data Analysis.pdf',
 'Hands-on-Machine-Learning.pdf']

In [ ]:
l= list()
for i in os.listdir(path):
    l.append(PyPDFLoader(os.path.join(path,i)))


docs = []
for loader in tqdm(l):
    docs.append(loader.load())

 48%|████▊     | 10/21 [01:00<00:52,  4.79s/it]Ignoring wrong pointing object 3268 0 (offset 0)
Ignoring wrong pointing object 3269 0 (offset 0)
Ignoring wrong pointing object 5594 0 (offset 0)
Ignoring wrong pointing object 5595 0 (offset 0)
Ignoring wrong pointing object 5596 0 (offset 0)
Ignoring wrong pointing object 5597 0 (offset 0)
Ignoring wrong pointing object 5598 0 (offset 0)
Ignoring wrong pointing object 5599 0 (offset 0)
Ignoring wrong pointing object 5600 0 (offset 0)
Ignoring wrong pointing object 5601 0 (offset 0)
Ignoring wrong pointing object 5602 0 (offset 0)
Ignoring wrong pointing object 5603 0 (offset 0)
Ignoring wrong pointing object 5604 0 (offset 0)
Ignoring wrong pointing object 5605 0 (offset 0)
Ignoring wrong pointing object 5606 0 (offset 0)
Ignoring wrong pointing object 5607 0 (offset 0)
Ignoring wrong pointing object 5608 0 (offset 0)
Ignoring wrong pointing object 5609 0 (offset 0)
Ignoring wrong pointing object 5610 0 (offset 0)
Ignoring wrong pointin

In [ ]:
docs[0][1].page_content

'\xa0\nTable of Contents \n\xa0\nTable\xa0of\xa0Contents\xa0\nIntroduction\xa0\nWhat\xa0is\xa0Data\xa0Science?\xa0\nDifferent\xa0Roles\xa0within\xa0Data\xa0Science\xa0\nHow\xa0Different\xa0Companies\xa0Think\xa0About\xa0Data\xa0Science\xa0\n1\xad\xa0Early\xadstage\xa0startups\xa0(200\xa0employees\xa0or\xa0fewer)\xa0looking\xa0to\xa0build\xa0a\xa0data\xa0product\xa0\n2\xad\xa0Early\xadstage\xa0startups\xa0(200\xa0employees\xa0or\xa0fewer)\xa0looking\xa0to\xa0take\xa0advantage\xa0of\xa0their\xa0data\xa0\n3\xad\xa0Mid\xadsize\xa0and\xa0large\xa0Fortune\xa0500\xa0companies\xa0who\xa0are\xa0looking\xa0to\xa0take\xa0advantage\xa0of\xa0their\xa0\ndata\xa0\n4\xad\xa0Large\xa0technology\xa0companies\xa0with\xa0well\xadestablished\xa0data\xa0teams\xa0\nIndustries\xa0that\xa0employ\xa0Data\xa0Scientists\xa0\nGetting\xa0a\xa0Data\xa0Science\xa0Interview\xa0\nNine\xa0Paths\xa0to\xa0a\xa0Data\xa0Science\xa0Interview\xa0\nTraditional\xa0Paths\xa0to\xa0Job\xa0Interviews\xa0\n1\xad\xa0Data\xa0Science\x

Chunking Data

In [ ]:
doc_splits = list()
# split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)
for doc in docs:
    doc_splits.extend(text_splitter.split_documents(doc))


In [ ]:
for i in range(len(doc_splits)):
    doc_splits[i].page_content = doc_splits[i].page_content.replace(u'\xa0',u' ')
    doc_splits[i].page_content = doc_splits[i].page_content.replace(u'\xad',u'-')
    doc_splits[i].page_content = doc_splits[i].page_content.replace('\u200b','*')
    doc_splits[i].page_content = doc_splits[i].page_content.replace('\n','')
    doc_splits[i].page_content = doc_splits[i].page_content.replace('\*','')

In [ ]:
for idx, split in enumerate(doc_splits):
    split.metadata["chunk"] = idx

print(f'Created {len(doc_splits):,} splits')

Created 32,561 splits


Display output of a chunk

In [ ]:
doc_splits[0]

Document(page_content='Table of Contents  Table of Contents Introduction What is Data Science? Different Roles within Data Science How Different Companies Think About Data Science 1- Early-stage startups (200 employees or fewer) looking to build a data product 2- Early-stage startups (200 employees or fewer) looking to take advantage of their data 3- Mid-size and large Fortune 500 companies who are looking to take advantage of their data 4- Large technology companies with well-established data teams', metadata={'source': '/Users/ashmitamukherjee/Documents/NvidiaRag/data/UltimateGuidetoDataScienceInterviews-2.pdf', 'page': 1, 'chunk': 0})

In [ ]:
doc_splits[0].page_content
#[f'content:-{doc.page_content} metadata:-{doc.metadata}'for doc in doc_splits]

'Table of Contents  Table of Contents Introduction What is Data Science? Different Roles within Data Science How Different Companies Think About Data Science 1- Early-stage startups (200 employees or fewer) looking to build a data product 2- Early-stage startups (200 employees or fewer) looking to take advantage of their data 3- Mid-size and large Fortune 500 companies who are looking to take advantage of their data 4- Large technology companies with well-established data teams'

Using Instruct Embeddings from HuggingFace for embedding the documents

In [ ]:
embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",model_kwargs = {'device': 'mps'},embed_instruction="These Documents will be used in a Question Answering context for an Educational Tool. Embed Accordingly.")
db = FAISS.from_documents([doc_splits[0]], embeddings)
with tqdm(total=len(doc_splits)-1, desc="Ingesting documents") as pbar:
    for doc in doc_splits[1:]:
            db.add_documents([doc])
            pbar.update(1)

load INSTRUCTOR_Transformer
max_seq_length  512


/opt/anaconda3/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:278: UserWarning: MPS: no support for int64 for sum_out_mps, downcasting to a smaller data type (int32/float32). Native support for int64 has been added in macOS 13.3. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:157.)
  assert torch.sum(attention_mask[local_idx]).item() >= context_masks[local_idx].item(),\
Ingesting documents: 100%|██████████| 32560/32560 [3:30:24<00:00,  2.58it/s]  


Saving and reloading the vector database to check if the embeddings are preserved

In [ ]:
db.save_local("faiss_index")

In [ ]:
new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

Using Llama3 through Ollama as the LLM for this RAG prcoess. I wanted to use open source + locally run LLM for this project.

In [ ]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="llama3",
    callback_manager=CallbackManager(
            [StreamingStdOutCallbackHandler()]
),
    repeat_penalty=1.15,
    stop=["<|eot_id|>"],
)

SImilarity Search

In [ ]:
query = 'What the K-means algorithm?'
docs = db.similarity_search(query)

page_content = docs[1].page_content.replace('\n', ' ')
source = docs[1].metadata['source']
page = docs[1].metadata['page']


print(f'Most relevant content: {page_content} \n in file: {source} \n on page: {page}')

Most relevant content: import numpy as npIntroducing k-MeansThe k-means algorithm searches for a predetermined number of clusters within anunlabeled multidimensional dataset. It accomplishes this using a simple conception ofwhat the optimal clustering looks like:•The “cluster center” is the arithmetic mean of all the points belonging to thecluster.•Each point is closer to its own cluster center than to other cluster centers. 
 in file: /Users/ashmitamukherjee/Documents/NvidiaRag/data/pythondatasciencehandbook.pdf 
 on page: 480


In [ ]:
query = 'what is the meaning of gradient descent?'
prompt = query
chain = load_qa_chain(
    llm,
    chain_type="stuff")

docs = db.similarity_search(query)
source = docs[0].metadata['source']
page = docs[0].metadata['page']
print(f'Found most relevant content in file: {source} on page: {page} \n')

response = chain.run(input_documents=docs, question=prompt)
print(response)

Found most relevant content in file: /Users/ashmitamukherjee/Documents/NvidiaRag/data/Hands-on-Machine-Learning.pdf on page: 144 

According to the context, Gradient Descent is a generic optimization algorithm that iteratively tweaks parameters to minimize a cost function by moving in the direction of the steepest descent along the error surface. It measures the local gradient of the error function with regards to the parameter vector and updates the parameters in the opposite direction of the gradient.According to the context, Gradient Descent is a generic optimization algorithm that iteratively tweaks parameters to minimize a cost function by moving in the direction of the steepest descent along the error surface. It measures the local gradient of the error function with regards to the parameter vector and updates the parameters in the opposite direction of the gradient.


Map Reduce Chain - Did not perform well. Reason : It is better at summarization tasks

In [ ]:
question_prompt_template = """Use the following portion of a long document to see if any of the text is relevant to answer the question.
Return all relevant text to the question asked.
{context}
Question: {question}
Relevant text, if any:"""

question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

combine_prompt_template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES").
If the information is not found in the document that was provided, indicate it is impossible to answer based on provided information,
abstain from answering and do not provide any document summary.
ALWAYS return a "SOURCES" part in your answer.

# QUESTION: {question}
# =========
# {summaries}
# =========
# FINAL ANSWER:"""

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries", "question"]
)

query = 'How to perform linear regression?'

chain = load_qa_with_sources_chain(
    llm,
    chain_type="map_reduce",
    return_intermediate_steps=True,
    question_prompt=question_prompt,
    combine_prompt=combine_prompt)

docs = db.similarity_search(query)

response = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

for n,item in enumerate(response['intermediate_steps']):
    print(f'Step #{n}')
    print(item)
    print('\n')

print(f'FINAL OUTPUT: {response["output_text"]}')

You've provided a snippet from a document!

According to your request, I'll search for relevant text within this portion of the document regarding "How to perform linear regression?"

Here's what I found:

"Figure 5-43. A linear regression modelIn Depth: Linear Regression | 391
Question: How to perform linear regression?
Relevant text, if any:"

Unfortunately, there isn't any relevant text provided in this snippet. The question is asked, but no answer or relevant information is given.

If you'd like me to help with anything else or search the surrounding context for answers, feel free to ask!The relevant text is:

We will start with the most familiar linear regression, a straight-line fit to data. Astraight-line fit is a model of the form y = ax + b where a is commonly known as the slope, and b is commonly known as the intercept.

This section explains the concept of simple linear regression and provides an example.Let's dive in!

According to the provided text, here's the relevant inf

Refine Chain - Best performing

In [ ]:
refine_template = (
    "The original question is as follows: {question}\n"
    "We have provided an existing answer, including sources: {existing_answer}\n"
    "We have the opportunity to refine the existing answer"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_str}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question"
    "If you do update it, please update the sources as well. "
    "If the context isn't useful, return the original answer."
    "Only display the final answer selected after completing ALL the iterations"
)
refine_prompt = PromptTemplate(
    input_variables=["question", "existing_answer", "context_str"],
    template=refine_template,
)


question_template = (
    "Context information is below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {question}\n"
)
question_prompt = PromptTemplate(
    input_variables=["context_str", "question"], template=question_template
)

query = 'can neural networks model any distribution?'
max_iterations= 3
chain = load_qa_with_sources_chain(
    llm,
    chain_type="refine",
    return_intermediate_steps=True,
    question_prompt=question_prompt,
    refine_prompt=refine_prompt)

docs = db.similarity_search(query)

response = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

#for n,item in enumerate(response['intermediate_steps']):
#    print(f'Step #{n}')
#    print(item)
#   print('\n')

print(f'FINAL OUTPUT: {response["output_text"]}')

Based on the provided context information, we can infer that the output layer of a neural network is capable of modeling complex distributions, including mixtures of Gaussian distributions. The text mentions that the neural network is able to learn non-linear mappings from the input to the parameters of the output distribution, which includes probabilities governing which mixture component will generate the output as well as the parameters for each mixture component.

This suggests that neural networks can model a wide range of distributions, including those with complex structures and multimodalities. In particular, the ability to learn the mixture proportions and component parameters allows the network to capture complex patterns in the data.

While it is possible to model some distributions using simple statistical models (e.g., normal or exponential), more complex distributions require more advanced modeling techniques. Neural networks have been shown to be capable of modeling a wi

Marginal Relevance search to look at the diversity of matched responses

In [ ]:
query='Tell me the names of the supervised algorithms in ml?Tell me the names and nothing else'
similarity_results = db.max_marginal_relevance_search(query,k=2, fetch_k=3, lambda_mult=0.4)
for n, item in enumerate(similarity_results):
    print(f'\033[1mMatch {n}:\033[0m {item.page_content}')

Match 0: categorized by model structure into supervised, unsupervised and reinforcement learning. With supervised ML algorithms we use labeled training data set to develop the model which
Match 1: The unsupervised learning algorithms that can be used for clustering are k-means, CURE (Clustering Using Representatives), SOM (Self-Organizing Map), Spectral clustering, LDA (Linear Discriminant Analysis). And evolutionary algorithms like genetic algorithm and particle swarm optimization is used.MACHINE LEARNING ALGORITHMS IN ONLINE TRAININGrtNEATReal-time Neuro-Evolution of Augmenting Topologies (rtNEAT) is the Neuro evolutionary algorithm


In [ ]:
from langchain.retrievers.document_compressors import LLMChainFilter
compressor = LLMChainFilter.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=db.as_retriever()
)
question = "What do you know about hypothesis tests?"
compressed_docs = compression_retriever.get_relevant_documents(question)

for n, doc in enumerate(compressed_docs):
    print(f'Document {n}: \n{doc.page_content} \nSource:{doc.metadata["source"]} \nPage:{doc.metadata["page"]} \n\n')

YESYESYESYESDocument 0: 
19.9 Hypothesis Testing 
Source:/Users/ashmitamukherjee/Documents/NvidiaRag/data/M2-Machine_Learning_By_Ethem_Alpaydin.pdf 
Page:535 


Document 1: 
Construct a hypothesisTesting of HypothesisThe Testing of hypothesis play a significant role in decision making in data analysis to determine whether the observed results follows certain assumption (hypothesis). 
Source:/Users/ashmitamukherjee/Documents/NvidiaRag/data/Handbook_of_Research_on_Emerging_Trends_and_Applications_of_Machine.pdf 
Page:237 


Document 2: 
136 Chapter 9. Hypothesis testing 
Source:/Users/ashmitamukherjee/Documents/NvidiaRag/data/thinkstats2.pdf 
Page:155 


Document 3: 
118 Chapter 9. Hypothesis testingThe goal of classical hypothesis testing is to answer the question, “Given asample and an apparent eﬀect, what is the probability of seeing such an eﬀectby chance?” Here’s how we answer that question:•The ﬁrst step is to quantify the size of the apparent eﬀect by choosing atest statistic . In

MultiQueryRetriver gives the most related queries related to the asked query!

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "What is hypothesis tests in statistics?"
#llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)
mqr = retriever_from_llm.invoke(question)


Here are three different versions of the original question:

What is the concept of hypothesis testing in statistics?

How do statisticians use hypothesis testing to draw conclusions about populations based on sample data?

Can you explain the purpose and process of conducting a statistical test, such as a null hypothesis test, in detail?

These alternative questions can help retrieve relevant documents from a vector database by capturing different aspects of the original question. For example:

* The first version focuses on the concept or definition of hypothesis testing, which may retrieve documents that provide a clear explanation of the term.
* The second version emphasizes the application of hypothesis testing in statistics, which may retrieve documents that discuss its use in drawing conclusions about populations.
* The third version delves deeper into the process and purpose of conducting a statistical test, which may retrieve documents that provide detailed information on spec

In [ ]:
retrievedDocs = await retriever_from_llm.aget_relevant_documents("How do statisticians use hypothesis testing to draw conclusions about populations based on sample data?")

Here are three alternative versions of the original question:

How do researchers utilize statistical hypothesis testing to make inferences about a larger population based on a representative sample?

What statistical techniques do analysts employ to validate hypotheses about a population's characteristics using information gathered from a subset of the overall dataset?

Can you provide examples or case studies illustrating how statisticians apply hypothesis testing to draw meaningful conclusions about populations, given limited data from a sample group?

RetrievalQA performs the best! However it is noticed that it doesn't have the ability to retain context which is why in the last section I have introduced ConversationalRetrievalChain for memory retention.

In [ ]:
from langchain.chains import RetrievalQA
from pprint import pprint

refine_prompt_template = """
You are given a question {question}

Based on this question, retrieve the context and give me more than 3 sentences. If you are unsure about the answer just say I dont have this information. Say thank you after answering!
{context}
"""
refine_prompt = PromptTemplate(
    template= refine_prompt_template,
    input_variables= ['question','context'],
)

qa_chain = RetrievalQA.from_chain_type(
    llm= llm,
    retriever=new_db.as_retriever(),
    #chain_type = 'refine',
    chain_type_kwargs = {"prompt":refine_prompt}
)
# Pass question to the qa_chain
question = "What is the generalizable error?"
result = qa_chain({"query": question})
#result["result"]

The context is about estimating the generalization error of a machine learning model, which refers to the expected value of the error on new inputs that were not part of the training set.

According to this text, the generalization error is defined as the expected value of the error on a new input. This expectation is taken across different possible inputs drawn from the distribution of inputs we expect the system to encounter in practice. The typical way to estimate the generalization error of a machine learning model is by measuring its performance on a test set that was collected separately from the training set.

In this context, it seems that the authors are discussing linear regression and stochastic gradient descent (SGD) as methods for minimizing the loss function with respect to the parameters. They also mention using a minibatch of examples to compute the gradient of the loss with respect to the parameters.

The generalization error can be estimated using the test set, but in

In [ ]:
question = "What is the irreducible error?"
result = qa_chain({"query": question})

The irreducible error refers to the noisiness of the data itself. This part of the error cannot be eliminated by increasing model complexity or training on more data, but rather requires careful data cleaning, such as fixing broken sensors or removing outliers. The only way to reduce this type of error is to improve the quality of the underlying data.

Thank you!

In [ ]:
question = "Describe Gini index"
result = qa_chain({"query": question})

Based on this context, I'll provide a detailed description of the Gini index:

The Gini index is a measure of impurity or uncertainty in a dataset. It's used to evaluate the quality of splits made during decision tree construction. The Gini index is defined as 1 minus the sum of the squared probabilities of each class in a node, where the probability of a class is the ratio of instances belonging to that class over the total number of instances.

In other words, given a dataset with n classes and m training instances, the Gini index for a particular node is calculated as:

Gi = 1 - ∑k=1npi,k2 \* pi,k

where pi,k is the ratio of class k instances among the training instances in the ith node.

The Gini index has several important properties. Firstly, it's maximized when classes are perfectly mixed (i.e., all classes have an equal number of instances). This means that if a decision tree splits a dataset into two child nodes with identical class distributions as the parent node, the Gini i

 Working of ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key='history',return_messages=True)


In [ ]:
memory1 = ConversationBufferMemory( memory_key="history")
memory1.save_context({"input": "hi"}, {"output": "whats up"})
print(f"memory1 ==> {memory1.load_memory_variables({})}")
memory2 = ConversationBufferMemory( memory_key="chat_history")
memory2.save_context({"input": "hi"}, {"output": "whats up"})
print(f"memory2 ==> {memory2.load_memory_variables({})}")
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "tell me more"}, {"output": "more"})
memory.load_memory_variables({})

memory1 ==> {'history': 'Human: hi\nAI: whats up'}
memory2 ==> {'chat_history': 'Human: hi\nAI: whats up'}


{'history': [HumanMessage(content='hi'),
  AIMessage(content='whats up'),
  HumanMessage(content='tell me more'),
  AIMessage(content='more'),
  HumanMessage(content='hi'),
  AIMessage(content='whats up'),
  HumanMessage(content='tell me more'),
  AIMessage(content='more')]}

In [ ]:
from langchain.chains import ConversationChain
conversation = ConversationChain (
    llm=llm,
    verbose = True,
    memory = ConversationBufferMemory(memory_key='history')
)
conversation.predict(input="Hi there!")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:
Nice to meet you! I'm glad you're initiating a chat with me. My training data consists of over 45 terabytes of text from various sources including books, articles, research papers, and websites. This extensive knowledge base allows me to provide accurate and informative responses to your queries. By the way, would you like to know more about my architecture or capabilities? I'm happy to share!
> Finished chain.


"Nice to meet you! I'm glad you're initiating a chat with me. My training data consists of over 45 terabytes of text from various sources including books, articles, research papers, and websites. This extensive knowledge base allows me to provide accurate and informative responses to your queries. By the way, would you like to know more about my architecture or capabilities? I'm happy to share!"

In [ ]:
from langchain.prompts.prompt import PromptTemplate
template = """You are a education question-answering assistant , answer questions that you know properly and concisely. If you don't know please say "I don't know".
Current conversation:
{history}
Human: {input}
AI:"""

PROMPT = PromptTemplate(input_variables=["history", "input"],
                        template=template)

conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory(human_prefix="Human")
                                )

In [ ]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
You are a education question-answering assistant , answer questions that you know properly and concisely. If you don't know please say "I don't know".
Current conversation:

Human: Hi there!
AI:
Hi there! I'm happy to help with any educational questions you may have. What's on your mind?
> Finished chain.


"Hi there! I'm happy to help with any educational questions you may have. What's on your mind?"

In [ ]:
conversation.predict(input="Do you know what is data science?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are a education question-answering assistant , answer questions that you know properly and concisely. If you don't know please say "I don't know".
Current conversation:
Human: Hi there!
AI: Hi there! I'm happy to help with any educational questions you may have. What's on your mind?
Human: Do you know what is data science?
AI:
Yes, I do! Data Science refers to the process of extracting insights and knowledge from large datasets using various techniques, including machine learning, statistical modeling, and data visualization. It involves using computational methods to analyze and interpret complex data in order to answer questions, identify patterns, and inform decision-making. Some common applications of data science include predicting customer behavior, detecting fraud, optimizing business processes, and making sense of vast amounts of data generated by sensors and other sources.
> Finished chain.


'Yes, I do! Data Science refers to the process of extracting insights and knowledge from large datasets using various techniques, including machine learning, statistical modeling, and data visualization. It involves using computational methods to analyze and interpret complex data in order to answer questions, identify patterns, and inform decision-making. Some common applications of data science include predicting customer behavior, detecting fraud, optimizing business processes, and making sense of vast amounts of data generated by sensors and other sources.'

Using ConversationalRetrievalChain we can see that the it retains memory. I have asked it multiple questions about Random Forest and it has maintained the context.

In [ ]:

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
template = """Given the following conversation and a follow up question, rephrase the follow up question to be a
standalone question without changing the content in given question. Make sure that the question is answered in 3 to 4 lines. If you dont know then say I dont know. Also say thanks for asking!

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
prompt_temp = PromptTemplate.from_template(template)


In [ ]:
question_generator = LLMChain(llm=llm, prompt=prompt_temp, memory=memory)
memory = ConversationBufferMemory(memory_key='chat_history',return_messages=True)
retriever_qa = new_db.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm = llm,
                                  retriever = retriever_qa,
                                  memory = memory,
                                  verbose= True)
question = "What is the use of Random Forest ?"
result_1 = qa({"question": question})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Random Forest is an ensemble technique that generates many decision trees and combines them to build a more realistic and accurate model. It can be used for classification as well as regression. Using scikit-learn in python, Random Forest can be used to list the relative importance of the features in classifying the target attribute. In this work, a group of 100 decision trees is used in random forest to give a better result. Since the student dropout data is class-imbalanced, so the class_

Random Forest Results Analysis on Various diseases:Random Forest is non linear ensemble classifier; we can apply on various clinical diseases identifica-tion and prediction. This algorithm used to reduce problem of over fitting. It c

In [ ]:
question = "What are its important hyperparameters? Give me around 5"
result_2 = qa({"question": question})




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the use of Random Forest ?
Assistant: According to the context, the uses of Random Forest include:

* Generating many decision trees and combining them to build a more realistic and accurate model
* Reducing the problem of over-fitting
* Classifying objects based on majority voting rules and ensemble of multiple decision trees
* Identifying and predicting various clinical diseases
* Providing a list of relative importance of features in classifying the target attribute

It is also mentioned that Random Forests are used to reduce variance, which suggests that it can be useful for improving model performance.
Human: Can it handle missing data?
Assistant: According to the provided context, I don't know if Random Forest can specifically handle missing data 

In [ ]:
question = "What kind of data does it work with?"
result_2 = qa({"question": question})




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the use of Random Forest ?
Assistant: According to the context, the uses of Random Forest include:

* Generating many decision trees and combining them to build a more realistic and accurate model
* Reducing the problem of over-fitting
* Classifying objects based on majority voting rules and ensemble of multiple decision trees
* Identifying and predicting various clinical diseases
* Providing a list of relative importance of features in classifying the target attribute

It is also mentioned that Random Forests are used to reduce variance, which suggests that it can be useful for improving model performance.
Human: Can it handle missing data?
Assistant: According to the provided context, I don't know if Random Forest can specifically handle missing data 

In [ ]:
question = "what is the algorithm of this model?"
result_3 = qa({"question": question})




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the use of Random Forest ?
Assistant: According to the context, the uses of Random Forest include:

* Generating many decision trees and combining them to build a more realistic and accurate model
* Reducing the problem of over-fitting
* Classifying objects based on majority voting rules and ensemble of multiple decision trees
* Identifying and predicting various clinical diseases
* Providing a list of relative importance of features in classifying the target attribute

It is also mentioned that Random Forests are used to reduce variance, which suggests that it can be useful for improving model performance.
Human: Can it handle missing data?
Assistant: According to the provided context, I don't know if Random Forest can specifically handle missing data 

In [ ]:
result_3

{'question': 'what is the algorithm of this model?',
 'chat_history': [HumanMessage(content='What is the use of Random Forest ?'),
  AIMessage(content='According to the context, the uses of Random Forest include:\n\n* Generating many decision trees and combining them to build a more realistic and accurate model\n* Reducing the problem of over-fitting\n* Classifying objects based on majority voting rules and ensemble of multiple decision trees\n* Identifying and predicting various clinical diseases\n* Providing a list of relative importance of features in classifying the target attribute\n\nIt is also mentioned that Random Forests are used to reduce variance, which suggests that it can be useful for improving model performance.'),
  HumanMessage(content='Can it handle missing data?'),
  AIMessage(content="According to the provided context, I don't know if Random Forest can specifically handle missing data or not. The text only discusses the general workings and characteristics of Random